# PCA and LDA Tutorial
This tutorial covers Principal Component Analysis (PCA) and Linear Discriminant Analysis (LDA), including standard usage with scikit-learn, manual calculation using Singular Value Decomposition (SVD), and data visualization techniques.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets import load_iris, fetch_olivetti_faces

## 1. PCA with Scikit-Learn
Principal Component Analysis identifies the orthogonal axes (principal components) that capture the maximum variance in the data.

In [ ]:
# Load dataset
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

# Fit PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Explained variance
explained_variance = pca.explained_variance_ratio_
print("Explained Variance Ratio (3 components):", explained_variance)
print("Total Variance Explained:", np.sum(explained_variance))

# Plot explained variance
plt.figure()
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
plt.title('Explained Variance by Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.xticks([1, 2, 3])
plt.show()

## 2. Projecting and Visualizing Data in 2D
Visualizing the high-dimensional dataset projected onto the first two principal components to assess natural class separation.

In [ ]:
plt.figure(figsize=(8, 6))
colors = ['red', 'green', 'blue']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], color=color, alpha=0.8, lw=lw, label=target_name)

plt.title('PCA of IRIS dataset (2D Projection)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.show()

## 3. Linear Discriminant Analysis (LDA)
Unlike PCA (unsupervised), LDA is a supervised dimensionality reduction technique. It utilizes class labels to maximize the ratio of between-class variance to within-class variance.

In [ ]:
# Fit LDA
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)

print("LDA Explained Variance Ratio:", lda.explained_variance_ratio_)

# Plot LDA projection
plt.figure(figsize=(8, 6))
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_lda[y == i, 0], X_lda[y == i, 1], alpha=0.8, color=color, label=target_name)

plt.title('LDA of IRIS dataset (2D Projection)')
plt.xlabel('Linear Discriminant 1')
plt.ylabel('Linear Discriminant 2')
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.show()

## 4. Manual PCA using SVD
Performing PCA from scratch requires centering the data and calculating the Singular Value Decomposition (SVD) of the centered matrix.

In [ ]:
# Center the data
X_mean = np.mean(X, axis=0)
X_centered = X - X_mean

# Compute SVD
# U: Left singular vectors, S: Singular values, Vt: Right singular vectors (principal axes)
U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)

# The principal components are the rows of Vt
# Project the data onto the first 2 principal components
W2 = Vt.T[:, :2]
X_manual_pca = X_centered.dot(W2)

# Verify manual PCA against scikit-learn
# Note: Principal component signs may be flipped depending on the SVD solver's initialization, 
# but the linear span remains identical.
print("First 5 rows of manual PCA:\n", X_manual_pca[:5])
print("\nFirst 5 rows of sklearn PCA (first 2 components):\n", X_pca[:5, :2])

## 5. High-Dimensional Image Data (Eigenfaces)
Applying PCA to image datasets for dimensionality reduction and feature extraction (creating the 'low-dimensional manifold' mentioned in the assignment).

In [ ]:
# Load a facial image dataset
faces = fetch_olivetti_faces(shuffle=True, random_state=42)
X_faces = faces.data
y_faces = faces.target

print("Original Faces Dataset Shape:", X_faces.shape) # (400, 4096)

# Apply PCA
n_components_faces = 150
pca_faces = PCA(n_components=n_components_faces, svd_solver='randomized', whiten=True).fit(X_faces)
eigenfaces = pca_faces.components_.reshape((n_components_faces, 64, 64))

print("PCA Faces Transformed Shape:", pca_faces.transform(X_faces).shape) # (400, 150)

# Visualize the first 4 eigenfaces
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, ax in enumerate(axes):
    ax.imshow(eigenfaces[i], cmap='gray')
    ax.set_title(f"Eigenface {i+1}")
    ax.axis('off')
plt.show()